In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

# print("Current EST Timestamp:", datetime.now(est).strftime("%Y-%m-%d %H:%M:%S %Z"))

dill 0.3.8 is installed but dill<0.3.2,>=0.3.1.1 is required
Redoing requirement with just package name...
cloudpickle 3.0.0 is installed but cloudpickle~=2.2.1 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<1.25.0,>=1.14.3 is required
Redoing requirement with just package name...
pyarrow 16.1.0 is installed but pyarrow<10.0.0,>=3.0.0 is required
Redoing requirement with just package name...
jupyterlab 4.2.5 is installed but jupyterlab~=3.6.0 is required
Redoing requirement with just package name...
google-cloud-bigquery 2.34.4 is installed but google-cloud-bigquery[bqstorage,pandas]>=3.10.0 is required
Redoing requirement with just package name...
google-cloud-storage 1.44.0 is installed but google-cloud-storage>=2.0.0 is required
Redoing requirement with just package name...
pandas 2.2.2 is installed but pandas<2.1.4,>=1.5.0 is required
Redoing requirement with just package name...
botocore 1.35.23 is installed but botocore<1.30.0,>=1.29

In [3]:
from datetime import datetime
import pytz
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
import torch
import pandas as pd

# Set timezone to EST
est = pytz.timezone('America/New_York')

# Get current date and time in EST
current_time_est = datetime.now(est)

# Print the timestamp in desired format
print("Current EST Timestamp:", datetime.now(est).strftime("%Y-%m-%d %H:%M:%S %Z"))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Current EST Timestamp: 2024-11-15 17:02:07 EST


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank, adjust as needed
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,  # Optimized to be 0
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.



### Question:
{}

### Answer:
{}

### Explainaition

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    output      = examples["is_correct"]
    texts = []
    for instruction, input, output in zip(question, ans, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [8]:
# Load and display the dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
train_dataset = dataset['train']

# total_size = len(dataset['train'])

# # Define the split index for the last 1000 rows
# train_size = total_size - 2000

# # Split the dataset
# train_dataset = dataset['train'].select(range(train_size))
# validation_dataset = dataset['train'].select(range(train_size, total_size))
# train_dataset = dataset.select(range(30000))
print("Current EST Timestamp:", datetime.now(est).strftime("%Y-%m-%d %H:%M:%S %Z"))


README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Current EST Timestamp: 2024-11-15 17:04:58 EST


In [9]:
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
# val_dataset = val_dataset.map(formatting_prompts_func, batched=True)
print("Current EST Timestamp:", datetime.now(est).strftime("%Y-%m-%d %H:%M:%S %Z"))

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Current EST Timestamp: 2024-11-15 17:05:13 EST


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [11]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    # num_train_epochs=max_epochs,
    warmup_steps = 100,
    learning_rate=2e-4,
    max_steps = 1500,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none"  # No checkpointing for tuning
    )

In [12]:
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    # eval_dataset=val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

Map (num_proc=4):   0%|          | 0/1000000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 1,500
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,2.226900
2,2.258400
3,2.363700
4,2.241200
5,2.406600
6,2.242000
7,2.056700
8,2.450300
9,2.275400
10,2.112600


TrainOutput(global_step=1500, training_loss=0.6204937378168106, metrics={'train_runtime': 14884.7679, 'train_samples_per_second': 1.612, 'train_steps_per_second': 0.101, 'total_flos': 2.1211612339401523e+17, 'train_loss': 0.6204937378168106, 'epoch': 0.024})

In [14]:
import pandas as pd
FastLanguageModel.for_inference(model)  # Enable fast inference
test_dataset = dataset['test']


# Prepare prompts for each question-answer pair
test_prompts = [prompt.format(q, a, "") for q, a in zip(test_dataset["question"], test_dataset["answer"])]

batch_size = 16  # Adjust batch size based on GPU memory constraints
predictions = []

# Perform inference in batches
for i in range(0, len(test_dataset), batch_size):
    batch_prompts = test_prompts[i:i + batch_size]
    
    # Tokenize the batch for generation
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")
    
    # Generate responses in batch
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the outputs and extract 'True' or 'False'
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for j, decoded_output in enumerate(decoded_outputs):
        # print(" ----- ")
        # print(decoded_outputs)
        # Check if 'True' or 'False' is in the response
        if decoded_output.strip().endswith("True"):
            is_correct = "True"
        elif decoded_output.strip().endswith("False"):
            is_correct = "False"
        else:
            is_correct = "False"  # Default to 'False' if ambiguous
        
        # Append prediction to the list
        predictions.append({"ID": i + j, "is_correct": is_correct})

    # Debug: Print progress
    print(f"Processed batch {i // batch_size + 1}/{len(test_prompts) // batch_size + 1} - Total predictions so far: {len(predictions)}")

# Convert predictions to DataFrame and save as CSV for the first 1000 rows
submission_df = pd.DataFrame(predictions)
submission_df['ID'] = range(len(submission_df))  # Sequential IDs for CSV
# print(submission_df)
# Save as CSV
submission_df.to_csv("submission_r64_a32_l_b4_a4_1500.csv", index=False)

print("Submission file 'submission_1000.csv' created with IDs from 0 to", len(submission_df) - 1)

Processed batch 1/626 - Total predictions so far: 16
Processed batch 2/626 - Total predictions so far: 32
Processed batch 3/626 - Total predictions so far: 48
Processed batch 4/626 - Total predictions so far: 64
Processed batch 5/626 - Total predictions so far: 80
Processed batch 6/626 - Total predictions so far: 96
Processed batch 7/626 - Total predictions so far: 112
Processed batch 8/626 - Total predictions so far: 128
Processed batch 9/626 - Total predictions so far: 144
Processed batch 10/626 - Total predictions so far: 160
Processed batch 11/626 - Total predictions so far: 176
Processed batch 12/626 - Total predictions so far: 192
Processed batch 13/626 - Total predictions so far: 208
Processed batch 14/626 - Total predictions so far: 224
Processed batch 15/626 - Total predictions so far: 240
Processed batch 16/626 - Total predictions so far: 256
Processed batch 17/626 - Total predictions so far: 272
Processed batch 18/626 - Total predictions so far: 288
Processed batch 19/626 - 

In [15]:
save_directory = "saved_model_test_3"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer have been saved to {save_directory}")


Model and tokenizer have been saved to saved_model_test_3


In [16]:
# # Assume FastLanguageModel and dataset are already defined
# FastLanguageModel.for_inference(model)  # Enable fast inference# Load your validation dataset

# # Prepare prompts for each question-answer pair in validation
# validation_prompts = [prompt.format(q, a, "") for q, a in zip(validation_dataset["question"], validation_dataset["answer"])]

# batch_size = 16  # Adjust batch size based on GPU memory constraints
# predictions = []
# correct = 0
# total = 0

In [17]:
# # Perform inference in batches
# for i in range(0, len(validation_dataset), batch_size):
#     batch_prompts = validation_prompts[i:i + batch_size]
    
#     # Tokenize the batch for generation
#     inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")
    
#     # Generate responses in batch
#     outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

#     # Decode the outputs and extract 'True' or 'False'
#     decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     # print(decoded_outputs)
#     for j, decoded_output in enumerate(decoded_outputs):
#         # Check if 'True' or 'False' is in the response
#         if decoded_output.strip().endswith("True"):
#             is_correct = "True"
#         elif decoded_output.strip().endswith("False"):
#             is_correct = "False"
#         else:
#             is_correct = "False"  # Default to 'False' if ambiguous
        
#         # Compare with actual label
#         actual_label = validation_dataset["is_correct"][i + j]
#         # print("actual_label is", actual_label)
#         # print("is_correct by model", is_correct)# Assuming 'correct_answer' is the true label
#         if str(is_correct) == str(actual_label):
#             correct += 1
#         total += 1

#         # Append prediction to the list
#         predictions.append({"ID": i + j, "is_correct": is_correct})

#     # Debug: Print progress
#     print(f"Processed batch {i // batch_size + 1}/{len(validation_prompts) // batch_size + 1} - Total predictions so far: {len(predictions)}")





In [18]:
# # Calculate accuracy
# accuracy = correct / total
# print(f"Validation Accuracy: {accuracy:.4f}")

In [19]:
# correct